# Inputs

In [1]:
splitPartCount = 5
splitSeed = 27
hoursPerWindow = 1

`functionize-notebook` has modified this notebook during execution. The following variables have been injected:

- hoursPerWindow: 1


# Preproccess

## read data

In [3]:
from utils.class_patient import Patients


patients = Patients.loadPatients()
len(patients)

1213

## remove missing

In [4]:
# fill measures whose null represent false value

from constants import NULLABLE_MEASURES


nullableMeasures = NULLABLE_MEASURES

for measureName in nullableMeasures:
    patients.fillMissingMeasureValue(measureName, 0)

In [5]:
# remove measures with less than 80% of data

measures = patients.getMeasures()

for measure, count in measures.items():
    if count < len(patients) * 80 / 100:
        patients.removeMeasures([measure])
        print(measure, count)

pco2 917
ph 954
po2 917
albumin 406
hba1c 326
lymphocyte 446
height 415
urine-ketone 294
crp 19


In [6]:
# remove patients with less than 80% of data

patients.removePatientByMissingFeatures()
len(patients)

1206

## split patients

In [7]:
splitedPatients = patients.split(splitPartCount, splitSeed)

len(splitedPatients[0])

242

In [8]:
from pandas import Timedelta


splitedPatients[0].getMeasuresBetween(Timedelta(-6), Timedelta(1))

,subject_id,hadm_id,stay_id,akd,ag,age,bg,bicarbonate,bun,calcium,...,rr,saps2,sbp,scr,sodium,sofa,use_NaHCO3,uti,wbc,weight
0,14866589,20066152,33060916,False,NaN,37,NaN,NaN,NaN,NaN,...,NaN,22,NaN,NaN,133.0,2,0.0,0,NaN,NaN
0,17922352,20070680,39895564,False,NaN,47,NaN,NaN,NaN,NaN,...,NaN,26,NaN,NaN,145.0,5,0.0,0,NaN,NaN
0,14849360,20152551,32817197,False,NaN,59,NaN,NaN,NaN,NaN,...,NaN,31,NaN,NaN,145.0,7,0.0,0,NaN,NaN
0,16815664,20240670,37751533,False,NaN,42,NaN,NaN,NaN,NaN,...,NaN,27,NaN,NaN,138.0,2,NaN,0,NaN,43.00
0,16792064,20244311,33026932,False,NaN,46,NaN,NaN,NaN,NaN,...,NaN,19,NaN,NaN,131.0,0,0.0,0,NaN,68.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,15517352,29801212,32814591,False,NaN,20,NaN,NaN,NaN,NaN,...,NaN,30,NaN,NaN,139.0,4,NaN,0,NaN,NaN
0,10383045,29899941,39755932,False,NaN,58,NaN,NaN,NaN,NaN,...,NaN,23,NaN,NaN,136.0,1,0.0,0,NaN,59.40
0,15679298,29905973,39834726,False,NaN,50,NaN,NaN,NaN,NaN,...,NaN,32,NaN,NaN,134.0,3,0.0,0,NaN,47.40
0,10912213,29911812,34040470,False,NaN,47,NaN,NaN,NaN,NaN,...,NaN,23,NaN,NaN,136.0,2,0.0,0,NaN,89.05


In [9]:
splitedPatients = patients.split(splitPartCount, splitSeed)


def trainTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        yield trainPatients, testPatients


def trainValTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        *trainPatients, valPatients = trainPatients.split(5, 27)
        tmpPatients = Patients(patients=[])
        for trainPatientsElem in trainPatients:
            tmpPatients += trainPatientsElem
        trainPatients = tmpPatients

        yield trainPatients, valPatients, testPatients

In [10]:
for trainPatients, testPatients in trainTest():
    print(len(trainPatients.patientList), len(testPatients.patientList))

964 242
965 241
965 241
965 241
965 241


In [11]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


dataset = []
for i, (trainPatients, testPatients) in enumerate(trainTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )
    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]

    dataset.append((npTrainX, trainY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [12]:
import pickle


(TEMP_PATH / f"train-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(dataset)
)

46548623

In [13]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


datavalset = []
for i, (trainPatients, valPatients, testPatients) in enumerate(trainValTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    npValX, *_ = patientsToNumpy(
        valPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]
    valY = [p.akdPositive for p in valPatients]

    datavalset.append((npTrainX, trainY, npValX, valY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [14]:
import pickle

(TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(datavalset)
)

45854217

# LSTM

In [15]:
import pickle
from constants import TEMP_PATH

# load if not exist dataset
if "dataset" not in locals():
    dataset = pickle.loads(
        (TEMP_PATH / f"dataset-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )
   

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["AUC", "accuracy", "precision", "recall"])
    return model

2024-06-07 22:36:19.053781: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-07 22:36:19.071654: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-06-07 22:36:19.341994: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npTestX, testY) in enumerate(dataset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        class_weight=weight,
        callbacks=[early_stopping],
    )

    testY = np.array(testY)
    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


2024-06-07 22:36:19.550845: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-07 22:36:19.565926: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/tu/codepy/hust.year2023.PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer usi

 1/25 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - AUC: 0.6167 - accuracy: 0.4062 - loss: 0.6789 - precision: 0.3704 - recall: 0.8333

11/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.6665 - accuracy: 0.5295 - loss: 0.6646 - precision: 0.4371 - recall: 0.8359

22/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.6860 - accuracy: 0.5769 - loss: 0.6563 - precision: 0.4797 - recall: 0.8086

...

14/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.9697 - accuracy: 0.9453 - loss: 0.2002 - precision: 0.9263 - recall: 0.9208 

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.9676 - accuracy: 0.9428 - loss: 0.2086 - precision: 0.9273 - recall: 0.9172 - val_AUC: 0.6325 - val_accuracy: 0.6684 - val_loss: 0.9733 - val_precision: 0.5667 - val_recall: 0.4722


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - AUC: 0.7969 - accuracy: 0.7188 - loss: 0.5630 - precision: 0.7059 - recall: 0.7500

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8007 - accuracy: 0.7116 - loss: 0.5689 - precision: 0.6221 - recall: 0.7759 


Loses: [0.5794881582260132, 0.5682457685470581, 0.5226597189903259, 0.572175145149231, 0.58930903673172] 0.5663755655288696 0.023004748250466917
AUCs: [0.7514142394065857, 0.7902735471725464, 0.8307498097419739, 0.7667267322540283, 0.7795602083206177] 0.7837449073791504 0.026849529105898714
Accuracies: [0.7355371713638306, 0.7759336233139038, 0.7759336233139038, 0.680497944355011, 0.6970954537391663] 0.7329995632171631 0.039341152876960245
Precisions: [0.6597937941551208, 0.6694915294647217, 0.6915887594223022, 0.5789473652839661, 0.5901639461517334] 0.6379970788955689 0.04497542151028702
Recals: [0.6736842393875122, 0.8404255509376526, 0.7789473533630371, 0.6947368383407593, 0.75789475440979] 0.7491377472877503 0.05991903142079726


## Static validate 

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["AUC", "accuracy", "precision", "recall"],
    )
    return model

In [19]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

if "datavalset" not in locals():
    datavalset = pickle.loads(
        (TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )

loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npValX, valY, npTestX, testY) in enumerate(datavalset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npValX = np.nan_to_num(npValX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    valY = np.array(valY)
    testY = np.array(testY)

    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_data=(npValX, valY),
        class_weight=weight,
        callbacks=[early_stopping],
    )

    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


 1/25 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - AUC: 0.4879 - accuracy: 0.5312 - loss: 0.6691 - precision: 0.3125 - recall: 0.5556

12/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.5624 - accuracy: 0.5570 - loss: 0.6821 - precision: 0.4372 - recall: 0.5927

24/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.6145 - accuracy: 0.5881 - loss: 0.6670 - precision: 0.4717 - recall: 0.6292

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - AUC: 0.6208 - accuracy: 0.5919 - loss: 0.6649 - precision: 0.4763 - recall: 0.6319 - val_AUC: 0.7971 - val_accuracy: 0.7240 - val_loss: 0.5758 - val_precision: 0.6122 - val_recall: 0.8000


...

14/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.9744 - accuracy: 0.9540 - loss: 0.1764 - precision: 0.9683 - recall: 0.9002 

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.9756 - accuracy: 0.9511 - loss: 0.1753 - precision: 0.9548 - recall: 0.9110 - val_AUC: 0.7358 - val_accuracy: 0.6995 - val_loss: 0.9483 - val_precision: 0.6098 - val_recall: 0.6579


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - AUC: 0.8379 - accuracy: 0.7188 - loss: 0.5141 - precision: 0.7059 - recall: 0.7500

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8108 - accuracy: 0.7036 - loss: 0.5463 - precision: 0.6137 - recall: 0.7724 


Loses: [0.5787121057510376, 0.5458759069442749, 0.5165859460830688, 0.606282114982605, 0.5727355480194092] 0.5640383243560791 0.03051917763105762
AUCs: [0.7747225761413574, 0.792951226234436, 0.8323720693588257, 0.7545421123504639, 0.7875270247459412] 0.7884230017662048 0.02564544584841499
Accuracies: [0.7355371713638306, 0.7385892271995544, 0.7385892271995544, 0.6846473217010498, 0.7012448310852051] 0.7197215557098389 0.02251089957136446
Precisions: [0.6504854559898376, 0.6476190686225891, 0.6290322542190552, 0.5811966061592102, 0.5920000076293945] 0.6200666785240173 0.028507677250014025
Recals: [0.7052631378173828, 0.7234042286872864, 0.821052610874176, 0.7157894968986511, 0.7789473533630371] 0.7488913655281066 0.04417999058151169
